# Proyecto 1 - Enviar el Estado del Tiempo por Mensaje con Twilio

In [1]:
import time
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from config import API_KEY_WAPI
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

### Cargar y Extraer los Datos de la Api

In [2]:
api_key = API_KEY_WAPI

query = "Barquisimeto"

weather_url = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'

In [3]:
response = requests.get(weather_url).json()

In [4]:
response

{'location': {'name': 'Barquisimeto',
  'region': 'Lara',
  'country': 'Venezuela',
  'lat': 10.07,
  'lon': -69.32,
  'tz_id': 'America/Caracas',
  'localtime_epoch': 1701200246,
  'localtime': '2023-11-28 15:37'},
 'current': {'last_updated_epoch': 1701199800,
  'last_updated': '2023-11-28 15:30',
  'temp_c': 30.0,
  'temp_f': 86.0,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 2.2,
  'wind_kph': 3.6,
  'wind_degree': 135,
  'wind_dir': 'SE',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.01,
  'precip_in': 0.0,
  'humidity': 46,
  'cloud': 75,
  'feelslike_c': 31.9,
  'feelslike_f': 89.4,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 6.0,
  'gust_mph': 13.9,
  'gust_kph': 22.4},
 'forecast': {'forecastday': [{'date': '2023-11-28',
    'date_epoch': 1701129600,
    'day': {'maxtemp_c': 29.6,
     'maxtemp_f': 85.2,
     'mintemp_c': 18.2,
     'mintemp_f': 64.7,
 

In [5]:
# obtener las keys del json
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [6]:
# ver las keys de forecast
response["forecast"]["forecastday"][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [7]:
# accedo a los values de las keys
response["forecast"]["forecastday"][0]["date"]

'2023-11-28'

In [8]:
# obtener la longitud de las horas del día
len(response["forecast"]["forecastday"][0]["hour"])

24

In [9]:
# obtener los values de hour
response["forecast"]["forecastday"][0]["hour"][1]["time"]

'2023-11-28 01:00'

In [10]:
# hacer split para obtener la fecha sin la hora
response["forecast"]["forecastday"][0]["hour"][1]["time"].split()[0]

'2023-11-28'

In [11]:
# hacer split para obtener la hora sin la fecha
response["forecast"]["forecastday"][0]["hour"][1]["time"].split()[1]

'01:00'

In [12]:
# hacer split para obtener la hora sola como número entero 
int(response["forecast"]["forecastday"][0]["hour"][1]["time"].split()[1].split(":")[0])

1

In [13]:
# obtener la condición
response["forecast"]["forecastday"][0]["hour"][0]["condition"]["text"]

'Clear'

In [14]:
# obtenemos la temperatura
response["forecast"]["forecastday"][0]["hour"][0]["temp_c"]

19.2

In [15]:
# obtenemos si lloverá o no: 0 = False, 1 = True
response["forecast"]["forecastday"][0]["hour"][0]["will_it_rain"]

0

In [16]:
# obtenemos la probabilidad de lluvia: sería un número entero hasta 100
response["forecast"]["forecastday"][0]["hour"][0]["chance_of_rain"]

0

### Armar el Dataframe

In [17]:
# función para obtener los datos de la api
def get_forecast(response, i):
    date_f = response["forecast"]["forecastday"][0]["hour"][i]["time"].split()[0]
    time_f = int(response["forecast"]["forecastday"][0]["hour"][i]["time"].split()[1].split(":")[0])
    condition_f = response["forecast"]["forecastday"][0]["hour"][i]["condition"]["text"]
    temp_f = response["forecast"]["forecastday"][0]["hour"][i]["temp_c"]
    rain_f = response["forecast"]["forecastday"][0]["hour"][i]["will_it_rain"]
    prob_rain_f = response["forecast"]["forecastday"][0]["hour"][i]["chance_of_rain"]

    return date_f, time_f, condition_f, temp_f, rain_f, prob_rain_f

In [18]:
# variable para llenar con el número de registros
datos = []

# ciclo para llenar el array
for i in tqdm(range(len(response["forecast"]["forecastday"][0]["hour"])), colour = "green"):
    datos.append(get_forecast(response, i))

100%|█████████████████████████████████████████████| 24/24 [00:00<00:00, 73530.53it/s]


In [19]:
datos

[('2023-11-28', 0, 'Clear', 19.2, 0, 0),
 ('2023-11-28', 1, 'Clear', 19.0, 0, 0),
 ('2023-11-28', 2, 'Clear', 18.9, 0, 0),
 ('2023-11-28', 3, 'Clear', 18.7, 0, 0),
 ('2023-11-28', 4, 'Clear', 18.5, 0, 0),
 ('2023-11-28', 5, 'Partly cloudy', 18.3, 0, 0),
 ('2023-11-28', 6, 'Partly cloudy', 18.2, 0, 0),
 ('2023-11-28', 7, 'Patchy rain possible', 18.4, 1, 84),
 ('2023-11-28', 8, 'Patchy rain possible', 22.6, 0, 63),
 ('2023-11-28', 9, 'Patchy rain possible', 25.2, 1, 80),
 ('2023-11-28', 10, 'Patchy rain possible', 26.8, 0, 62),
 ('2023-11-28', 11, 'Patchy rain possible', 28.0, 0, 61),
 ('2023-11-28', 12, 'Patchy rain possible', 28.8, 1, 78),
 ('2023-11-28', 13, 'Patchy rain possible', 29.3, 1, 81),
 ('2023-11-28', 14, 'Patchy rain possible', 29.6, 1, 72),
 ('2023-11-28', 15, 'Partly cloudy', 30.0, 1, 76),
 ('2023-11-28', 16, 'Patchy rain possible', 28.5, 1, 83),
 ('2023-11-28', 17, 'Sunny', 27.4, 0, 0),
 ('2023-11-28', 18, 'Patchy rain possible', 24.5, 1, 85),
 ('2023-11-28', 19, 'Patchy

In [20]:
# ver los datos para una hora específica - el array tiene 24 registros(0-23), que son las horas del día
datos[9]

('2023-11-28', 9, 'Patchy rain possible', 25.2, 1, 80)

In [21]:
# columnas
cols = ['Fecha', 'Hora', 'Condición', 'Temperatura', 'Lluvia', 'Prob_lluvia']

# crear el dataframe
df = pd.DataFrame(datos, columns=cols)

# ver el dataframe
df

,Fecha,Hora,Condición,Temperatura,Lluvia,Prob_lluvia
0,2023-11-28,0,Clear,19.2,0,0
1,2023-11-28,1,Clear,19.0,0,0
2,2023-11-28,2,Clear,18.9,0,0
3,2023-11-28,3,Clear,18.7,0,0
4,2023-11-28,4,Clear,18.5,0,0
5,2023-11-28,5,Partly cloudy,18.3,0,0
6,2023-11-28,6,Partly cloudy,18.2,0,0
7,2023-11-28,7,Patchy rain possible,18.4,1,84
8,2023-11-28,8,Patchy rain possible,22.6,0,63
9,2023-11-28,9,Patchy rain possible,25.2,1,80


In [22]:
df[df["Lluvia"] == 1]
df[df["Prob_lluvia"] != 0]

,Fecha,Hora,Condición,Temperatura,Lluvia,Prob_lluvia
7,2023-11-28,7,Patchy rain possible,18.4,1,84
8,2023-11-28,8,Patchy rain possible,22.6,0,63
9,2023-11-28,9,Patchy rain possible,25.2,1,80
10,2023-11-28,10,Patchy rain possible,26.8,0,62
11,2023-11-28,11,Patchy rain possible,28.0,0,61
12,2023-11-28,12,Patchy rain possible,28.8,1,78
13,2023-11-28,13,Patchy rain possible,29.3,1,81
14,2023-11-28,14,Patchy rain possible,29.6,1,72
15,2023-11-28,15,Partly cloudy,30.0,1,76
16,2023-11-28,16,Patchy rain possible,28.5,1,83


In [23]:
# crear un dataframe filtrando los resultados por lluvia y hora
df_rain = df[(df["Lluvia"] == 1) & (df["Hora"] > 6) & (df["Hora"] < 22)]
df_rain = df_rain[["Hora", "Condición"]]
df_rain.set_index("Hora", inplace=True)

In [24]:
df_rain

,Condición
Hora,
7,Patchy rain possible
9,Patchy rain possible
12,Patchy rain possible
13,Patchy rain possible
14,Patchy rain possible
15,Partly cloudy
16,Patchy rain possible
18,Patchy rain possible
19,Patchy rain possible


### Mensaje Template

In [25]:
# mensaje a imprimir para informar el pronóstico
print('\n¡Hola! \n\n El pronóstico del tiempo hoy ' + df["Fecha"][0] + ' en ' + query + ' es: \n\n ' + str(df_rain))
# '\n¡Hola! \n\n El pronóstico del tiempo hoy ' + df["Fecha"][0] + ' en ' + query + ' es: \n\n ' + str(df_rain)



¡Hola! 

 El pronóstico del tiempo hoy 2023-11-28 en Barquisimeto es: 

                  Condición
Hora                      
7     Patchy rain possible
9     Patchy rain possible
12    Patchy rain possible
13    Patchy rain possible
14    Patchy rain possible
15           Partly cloudy
16    Patchy rain possible
18    Patchy rain possible
19    Patchy rain possible
20    Patchy rain possible
21    Patchy rain possible
